## Part 1: Preprocessing

In [20]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [21]:
# Determine the number of unique values in each column
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [22]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [23]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'BusinessTravel', 'Department', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction', 'TotalWorkingYears', 'HourlyRate', 'JobInvolvement', 'YearsAtCompany']


# Create X_df using your selected columns
X_df = attrition_df[selected_columns]


# Show the data types for X_df
X_df.dtypes


,0
Age,int64
BusinessTravel,object
Department,object
DistanceFromHome,int64
Education,int64
EnvironmentSatisfaction,int64
TotalWorkingYears,int64
HourlyRate,int64
JobInvolvement,int64
YearsAtCompany,int64


In [24]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=37)


In [25]:
# Convert your X data to numeric data types however you see fit
X_train = pd.get_dummies(X_train)
print(X_train.dtypes)


Age                                  int64
DistanceFromHome                     int64
Education                            int64
EnvironmentSatisfaction              int64
TotalWorkingYears                    int64
HourlyRate                           int64
JobInvolvement                       int64
YearsAtCompany                       int64
BusinessTravel_Non-Travel             bool
BusinessTravel_Travel_Frequently      bool
BusinessTravel_Travel_Rarely          bool
Department_Human Resources            bool
Department_Research & Development     bool
Department_Sales                      bool
dtype: object


In [26]:
X_test = pd.get_dummies(X_test)
print(X_test.dtypes)

Age                                  int64
DistanceFromHome                     int64
Education                            int64
EnvironmentSatisfaction              int64
TotalWorkingYears                    int64
HourlyRate                           int64
JobInvolvement                       int64
YearsAtCompany                       int64
BusinessTravel_Non-Travel             bool
BusinessTravel_Travel_Frequently      bool
BusinessTravel_Travel_Rarely          bool
Department_Human Resources            bool
Department_Research & Development     bool
Department_Sales                      bool
dtype: object


In [27]:
# Create a StandardScaler
scaler = StandardScaler()


# Fit the StandardScaler to the training data
scaler.fit(X_train)


# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [28]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
OHE = OneHotEncoder()


# Fit the encoder to the training data
OHE.fit(y_train[['Department']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded = OHE.transform(y_train[['Department']]).toarray()
y_test_encoded = OHE.transform(y_test[['Department']]).toarray()


In [29]:
# Create a OneHotEncoder for the Attrition column
OHE = OneHotEncoder()


# Fit the encoder to the training data
OHE.fit(y_train[['Attrition']])


# Create two new variables by applying the encoder
# to the training and testing data
y_train_encoded_attrition = OHE.transform(y_train[['Attrition']]).toarray()
y_test_encoded_attrition = OHE.transform(y_test[['Attrition']]).toarray()


## Part 2: Create, Compile, and Train the Model

In [30]:
# Find the number of columns in the X training data.
input_dim = X_train_scaled.shape[1]


# Create the input layer
input_layer = layers.Input(shape=(input_dim,))


# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)


In [31]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(16, activation='relu')(shared_layer2)


# Create the output layer
department_output_layer = layers.Dense(3, activation='softmax', name='department_output')(department_hidden_layer)

In [32]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(16, activation='relu')(shared_layer2)


# Create the output layer
attrition_output_layer = layers.Dense(2, activation='sigmoid', name='attrition_output')(attrition_hidden_layer)


In [38]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})


# Summarize the model
model.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 14)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 64)             │            960 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 32)             │          2,080 │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 16)             │            528 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 16)             │            528 │ dense_5[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             51 │ dense_6[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             34 │ dense_7[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,181 (16.33 KB)

 Trainable params: 4,181 (16.33 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:
# Train the model
new_model = model.fit(
    X_train_scaled,
    {'department_output': y_train_encoded, 'attrition_output': y_train_encoded_attrition},
    epochs=50,
    batch_size=32)



Epoch 1/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.9753 - attrition_output_loss: 0.0787 - department_output_accuracy: 1.0000 - department_output_loss: 1.3989e-06 - loss: 0.0786
Epoch 2/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.9652 - attrition_output_loss: 0.0988 - department_output_accuracy: 1.0000 - department_output_loss: 1.0791e-06 - loss: 0.0987
Epoch 3/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.9763 - attrition_output_loss: 0.0779 - department_output_accuracy: 1.0000 - department_output_loss: 1.4272e-06 - loss: 0.0779
Epoch 4/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.9783 - attrition_output_loss: 0.0773 - department_output_accuracy: 1.0000 - department_output_loss: 9.4829e-07 - loss: 0.0773
Epoch 5/50
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_output_accuracy: 0.9816 - attrition_output_loss: 0.0726 - department_output_accuracy: 1.0000 - department_outp

In [43]:
# Evaluate the model with the testing data
test_loss, department_loss, attrition_loss, department_accuracy, attrition_accuracy = model.evaluate(X_test_scaled, {'department_output': y_test_encoded, 'attrition_output': y_test_encoded_attrition})


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.7294 - attrition_output_loss: 1.5953 - department_output_accuracy: 1.0000 - department_output_loss: 4.4401e-07 - loss: 1.6041 


In [46]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {department_accuracy}")
print(f"Attrition Accuracy: {attrition_accuracy}")


Department Accuracy: 0.7364130616188049
Attrition Accuracy: 1.0


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

I would say that accuracy is a common metric that we ahve learned in classes for evaluating model it might not be always our best options in case if we have imbalanced datasets but for the results I got, I think accuracy could be the best metric.
2. What activation functions did you choose for your output layers, and why?

The activation function that I used for the output layer is sigmoid, why sigmoid? I used sigmoid due to we use sigmoid for binary classification problem as we're creating a model for the HR model where will predict whether an employee are likely to leave the company.

3. Can you name a few ways that this model might be improved?

I think that we could try to improve our model with more data to improve the deparment accuracy score but also as the attrition accuracy is 100% we might take in consideration that when a model is working at perfection the model might be overfiting by reducing complexity.
also we can used data augmentation if the given data is not enought to train our model to make better predictions



YOUR ANSWERS HERE

1.
2.
3.